In [15]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow

### download and clean data

In [16]:
con_data = pd.read_csv('https://cocl.us/concrete_data')
con_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [17]:
con_data.shape

(1030, 9)

In [18]:
con_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [19]:
con_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

#### data cleaned

In [20]:
con_data_column=con_data.columns
X = con_data[con_data_column[con_data_column!= 'Strength']]
y = con_data['Strength']
X.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [21]:
y.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

### inputs of NN

In [22]:
n_cols= X.shape[1]

# Part B: Normalize the data

### A.1: Split the data

In [23]:
from sklearn.model_selection import train_test_split
# split after, in train_and_test function

### A.2: The regression model

In [24]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

def regression_model(network_arch, n_cols, optimizer='adam', loss='mean_squared_error'):
    '''
    network_arch: 
          {"layer_1": {"n_nodes": n1, "activation": activation1},
           ...
           "layer_n": {"n_nodes: nn}
          }
    '''
    model = Sequential()
    n_layers = len(network_arch)
    output_layer = "layer_"+str(n_layers)
    
    for i,layer in network_arch.items():
        if i == "layer_1":
            model.add(Dense(int(layer.get("n_nodes")), activation=layer.get("activation"), input_shape=(n_cols,)))
        elif i == output_layer:
            model.add(Dense(int(layer.get("n_nodes"))))
        else:
            model.add(Dense(int(layer.get("n_nodes")), activation=layer.get("activation")))
    model.compile(optimizer=optimizer, loss=loss)
    return model

In [28]:
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

def train_and_test(X, y, network_arch, n_cols, optimizer , loss, n_times, epochs, norm, seed=360):
    '''
    seed: for spilt data
    norm: (bool) normalization or not
    '''
    mse_list=[]
    for i in range(n_times):
        if norm:
            X= preprocessing.StandardScaler().fit(X).transform(X)
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed+i)
        model = regression_model(network_arch=network_arch, n_cols=n_cols)
        model.fit(X_train, y_train, epochs= epochs, verbose=2)
        y_hat= model.predict(X_test)
        mse = mean_squared_error(y_test, y_hat)
        print("time {} : mse= {}".format(i+1,mse))
        mse_list.append(mse)
    return mse_list

### B.3: Part B result


In [26]:
model_archA = {"layer_1": {"n_nodes": 10, "activation": "relu"}, "layer_2": {"n_nodes": 1},
                }

In [29]:
mse_listA=train_and_test(X=X, y=y, network_arch=model_archA, n_cols=n_cols, optimizer="adam", loss="mean_squared_error", 
                        n_times=50, epochs=50, norm= True, seed=360)

Epoch 1/50
23/23 - 1s - loss: 1564.1973 - 1s/epoch - 45ms/step
Epoch 2/50
23/23 - 0s - loss: 1551.4320 - 58ms/epoch - 3ms/step
Epoch 3/50
23/23 - 0s - loss: 1538.7361 - 44ms/epoch - 2ms/step
Epoch 4/50
23/23 - 0s - loss: 1525.7538 - 45ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 1512.4038 - 44ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 1498.3403 - 44ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 1483.3593 - 40ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 1467.3131 - 46ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 1450.1018 - 40ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 1431.5710 - 41ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 1411.7352 - 46ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 1390.4581 - 47ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 1368.3052 - 44ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 1344.4846 - 40ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 1319.4720 - 41ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 1293

Epoch 28/50
23/23 - 0s - loss: 816.7537 - 46ms/epoch - 2ms/step
Epoch 29/50
23/23 - 0s - loss: 783.6451 - 45ms/epoch - 2ms/step
Epoch 30/50
23/23 - 0s - loss: 750.5641 - 45ms/epoch - 2ms/step
Epoch 31/50
23/23 - 0s - loss: 718.1839 - 43ms/epoch - 2ms/step
Epoch 32/50
23/23 - 0s - loss: 686.0485 - 43ms/epoch - 2ms/step
Epoch 33/50
23/23 - 0s - loss: 654.4991 - 39ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 623.5131 - 44ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 593.4827 - 45ms/epoch - 2ms/step
Epoch 36/50
23/23 - 0s - loss: 564.4896 - 44ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 536.5704 - 42ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 509.7296 - 42ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 484.1948 - 42ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 459.6985 - 42ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 436.7023 - 46ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 415.5051 - 47ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 395.0232 

Epoch 4/50
23/23 - 0s - loss: 1539.1906 - 43ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 1519.8058 - 44ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 1500.5004 - 46ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 1481.2544 - 48ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 1461.8374 - 47ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 1442.5380 - 50ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 1422.6306 - 58ms/epoch - 3ms/step
Epoch 11/50
23/23 - 0s - loss: 1402.5826 - 48ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 1382.1820 - 48ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 1361.5599 - 46ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 1340.0970 - 43ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 1318.7100 - 41ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 1296.6626 - 38ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 1274.0933 - 38ms/epoch - 2ms/step
Epoch 18/50
23/23 - 0s - loss: 1250.9402 - 48ms/epoch - 2ms/step
Epoch 19/50
23/23 - 0s - loss: 

Epoch 31/50
23/23 - 0s - loss: 798.5992 - 50ms/epoch - 2ms/step
Epoch 32/50
23/23 - 0s - loss: 769.3927 - 46ms/epoch - 2ms/step
Epoch 33/50
23/23 - 0s - loss: 740.6500 - 45ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 712.2291 - 48ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 684.5195 - 49ms/epoch - 2ms/step
Epoch 36/50
23/23 - 0s - loss: 657.5322 - 49ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 631.1147 - 49ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 605.3638 - 49ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 580.1431 - 51ms/epoch - 2ms/step
Epoch 40/50
23/23 - 0s - loss: 556.1179 - 51ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 532.5740 - 47ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 510.5385 - 50ms/epoch - 2ms/step
Epoch 43/50
23/23 - 0s - loss: 488.9141 - 49ms/epoch - 2ms/step
Epoch 44/50
23/23 - 0s - loss: 468.4819 - 52ms/epoch - 2ms/step
Epoch 45/50
23/23 - 0s - loss: 449.0414 - 48ms/epoch - 2ms/step
Epoch 46/50
23/23 - 0s - loss: 429.9817 

Epoch 7/50
23/23 - 0s - loss: 1503.9448 - 44ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 1487.4995 - 43ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 1470.1790 - 41ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 1451.9252 - 40ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 1432.7142 - 43ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 1412.0498 - 38ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 1390.4178 - 41ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 1367.2938 - 39ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 1342.7086 - 38ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 1316.7095 - 40ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 1289.1628 - 38ms/epoch - 2ms/step
Epoch 18/50
23/23 - 0s - loss: 1260.5199 - 39ms/epoch - 2ms/step
Epoch 19/50
23/23 - 0s - loss: 1230.0315 - 41ms/epoch - 2ms/step
Epoch 20/50
23/23 - 0s - loss: 1198.4592 - 42ms/epoch - 2ms/step
Epoch 21/50
23/23 - 0s - loss: 1165.5957 - 38ms/epoch - 2ms/step
Epoch 22/50
23/23 - 0s - los

...

Epoch 48/50
23/23 - 0s - loss: 290.1397 - 40ms/epoch - 2ms/step
Epoch 49/50
23/23 - 0s - loss: 280.4700 - 40ms/epoch - 2ms/step
Epoch 50/50
23/23 - 0s - loss: 271.5495 - 41ms/epoch - 2ms/step
time 38 : mse= 287.37494504137237
Epoch 1/50
23/23 - 1s - loss: 1581.6414 - 653ms/epoch - 28ms/step
Epoch 2/50
23/23 - 0s - loss: 1564.7327 - 43ms/epoch - 2ms/step
Epoch 3/50
23/23 - 0s - loss: 1548.2281 - 42ms/epoch - 2ms/step
Epoch 4/50
23/23 - 0s - loss: 1531.6262 - 45ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 1514.9724 - 38ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 1497.3721 - 36ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 1479.3835 - 46ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 1460.5752 - 42ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 1441.0238 - 38ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 1420.5818 - 34ms/epoch - 1ms/step
Epoch 11/50
23/23 - 0s - loss: 1399.4789 - 40ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 1377.3475 - 35ms/epoch - 2ms/step
E

Epoch 24/50
23/23 - 0s - loss: 1001.6738 - 35ms/epoch - 2ms/step
Epoch 25/50
23/23 - 0s - loss: 962.5083 - 35ms/epoch - 2ms/step
Epoch 26/50
23/23 - 0s - loss: 923.7940 - 35ms/epoch - 2ms/step
Epoch 27/50
23/23 - 0s - loss: 885.2791 - 36ms/epoch - 2ms/step
Epoch 28/50
23/23 - 0s - loss: 847.0659 - 35ms/epoch - 2ms/step
Epoch 29/50
23/23 - 0s - loss: 809.6500 - 36ms/epoch - 2ms/step
Epoch 30/50
23/23 - 0s - loss: 773.1460 - 35ms/epoch - 2ms/step
Epoch 31/50
23/23 - 0s - loss: 736.9604 - 35ms/epoch - 2ms/step
Epoch 32/50
23/23 - 0s - loss: 701.8112 - 34ms/epoch - 1ms/step
Epoch 33/50
23/23 - 0s - loss: 668.2156 - 38ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 634.9891 - 37ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 603.5342 - 34ms/epoch - 1ms/step
Epoch 36/50
23/23 - 0s - loss: 573.1736 - 35ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 544.0966 - 35ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 516.3603 - 34ms/epoch - 1ms/step
Epoch 39/50
23/23 - 0s - loss: 490.5489

time 43 : mse= 294.4716519485186
Epoch 1/50
23/23 - 1s - loss: 1569.3064 - 657ms/epoch - 29ms/step
Epoch 2/50
23/23 - 0s - loss: 1553.4913 - 37ms/epoch - 2ms/step
Epoch 3/50
23/23 - 0s - loss: 1537.4624 - 38ms/epoch - 2ms/step
Epoch 4/50
23/23 - 0s - loss: 1521.0887 - 48ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 1504.2034 - 36ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 1486.5128 - 36ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 1467.9062 - 37ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 1448.1049 - 43ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 1427.2363 - 35ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 1405.1952 - 35ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 1382.0280 - 37ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 1357.6237 - 35ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 1331.9971 - 36ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 1305.3575 - 36ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 1277.7756 - 35ms/epoch - 2ms/step

Epoch 27/50
23/23 - 0s - loss: 1162.9517 - 35ms/epoch - 2ms/step
Epoch 28/50
23/23 - 0s - loss: 1142.4702 - 35ms/epoch - 2ms/step
Epoch 29/50
23/23 - 0s - loss: 1121.6332 - 35ms/epoch - 2ms/step
Epoch 30/50
23/23 - 0s - loss: 1100.7947 - 35ms/epoch - 2ms/step
Epoch 31/50
23/23 - 0s - loss: 1078.9491 - 40ms/epoch - 2ms/step
Epoch 32/50
23/23 - 0s - loss: 1057.4180 - 36ms/epoch - 2ms/step
Epoch 33/50
23/23 - 0s - loss: 1035.2561 - 36ms/epoch - 2ms/step
Epoch 34/50
23/23 - 0s - loss: 1012.9482 - 42ms/epoch - 2ms/step
Epoch 35/50
23/23 - 0s - loss: 990.2242 - 36ms/epoch - 2ms/step
Epoch 36/50
23/23 - 0s - loss: 967.2321 - 36ms/epoch - 2ms/step
Epoch 37/50
23/23 - 0s - loss: 943.9983 - 36ms/epoch - 2ms/step
Epoch 38/50
23/23 - 0s - loss: 920.2178 - 35ms/epoch - 2ms/step
Epoch 39/50
23/23 - 0s - loss: 896.3047 - 34ms/epoch - 1ms/step
Epoch 40/50
23/23 - 0s - loss: 871.9882 - 35ms/epoch - 2ms/step
Epoch 41/50
23/23 - 0s - loss: 847.8217 - 42ms/epoch - 2ms/step
Epoch 42/50
23/23 - 0s - loss: 8

Epoch 3/50
23/23 - 0s - loss: 1486.3656 - 44ms/epoch - 2ms/step
Epoch 4/50
23/23 - 0s - loss: 1468.0964 - 42ms/epoch - 2ms/step
Epoch 5/50
23/23 - 0s - loss: 1448.7402 - 43ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 1428.6403 - 42ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 1407.5205 - 44ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 1385.5510 - 42ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 1361.9203 - 43ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 1337.9504 - 43ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 1312.3978 - 42ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 1285.7646 - 44ms/epoch - 2ms/step
Epoch 13/50
23/23 - 0s - loss: 1258.5251 - 42ms/epoch - 2ms/step
Epoch 14/50
23/23 - 0s - loss: 1229.8531 - 44ms/epoch - 2ms/step
Epoch 15/50
23/23 - 0s - loss: 1200.6622 - 44ms/epoch - 2ms/step
Epoch 16/50
23/23 - 0s - loss: 1170.5920 - 45ms/epoch - 2ms/step
Epoch 17/50
23/23 - 0s - loss: 1139.2787 - 46ms/epoch - 2ms/step
Epoch 18/50
23/23 - 0s - loss: 1

In [33]:
print(mse_listA)

[344.4444509565407, 334.1563468746655, 266.25095178495894, 329.02753903217217, 275.7507970054131, 516.6835025948973, 390.39708622110874, 398.22592718656716, 267.4574611287606, 316.0282867404746, 294.3967642059638, 360.9758613144723, 317.09953056343704, 287.27527674297295, 299.0123202320403, 243.38648256434664, 625.4541025329428, 216.1730619982358, 358.80631183396014, 402.68593292843303, 479.4855729455432, 292.7265442365663, 378.71291757604655, 253.1149110015093, 347.5705354226765, 347.67534473296524, 322.8310548959166, 356.94779375212045, 379.86014554627513, 358.59037911466805, 318.9135173602223, 324.9694418955906, 411.8304239757091, 329.7513635610762, 478.59561690958054, 278.3718423281869, 431.86812591336746, 287.37494504137237, 428.52328827083846, 316.9612411263604, 251.42147803450885, 439.183882097439, 294.4716519485186, 352.6629304749519, 367.92435163444, 618.9423949969522, 345.19252102665274, 269.16181486852895, 296.7609758361896, 291.791602757886]


In [31]:
df_results = pd.DataFrame.from_dict({"Part": ["B"], "mean mse": [np.mean(mse_listA)], "std mse": [np.std(mse_listA)]})
df_results

,Part,mean mse,std mse
0,B,349.917532,84.306557


## Part B result is better than Part A